In [ ]:
# Projeto 1 de Grafos 
## Disciplina: Teoria dos Grafos — UnB
## Turma: 01, 2025/2
## Professor: Dibio
### Integrantes:
#- Julia Paulo Amorim - 241039270
#- Leticia Gonçalves Bomfim - 241002411
#- Vitor Alencar Ribeiro - 231036292

#Etapa 1 - Coleta de Dados:
